In [3]:
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('C:\\Users\\Bestc\\Desktop\\빅데이터 지능형 서비스 개발과정(멀티캠퍼스)\\설치프로그램\\Selenium\\chromedriver.exe', options = options)
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)
trs = driver.find_elements_by_tag_name('.aos-init')
len(trs)

100

In [18]:
from bs4 import BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [23]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
print(category, type(category))
name = channel.select_one('.subject a').text.strip()
print(name, type(name))
subscriber = channel.select_one('.subscriber_cnt').text
print(subscriber, type(subscriber))
view = channel.select_one('.view_cnt').text
print(view, type(view))
video = channel.select_one('.video_cnt').text[:-1]
print(video, type(video))

category_list, name_list, subscriber_list, view_list, video_list = [], [] ,[] , [], []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    category_list.append(category)
    name_list.append(name)
    subscriber_list.append(subscriber)
    view_list.append(view)
    video_list.append(video)
df = pd.DataFrame({
    '카테고리' : category_list, '이름' : name_list, '구독자 수' : subscriber_list, '조회수' : view_list, '영상 수' : video_list
})
df

[음악/댄스/가수] <class 'str'>
BLACKPINK <class 'str'>
6400만 <class 'str'>
190억0381만 <class 'str'>
371 <class 'str'>


,카테고리,이름,구독자 수,조회수,영상 수
0,[음악/댄스/가수],BLACKPINK,6400만,190억0381만,371
1,[음악/댄스/가수],HYBE LABELS,6030만,187억1305만,654
2,[음악/댄스/가수],BANGTANTV,5640만,121억9496만,"1,579"
3,[음악/댄스/가수],SMTOWN,2850만,218억7092만,"3,729"
4,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223
...,...,...,...,...,...
95,[음식/요리/레시피],까니짱 [ G-NI ],408만,10억3471만,482
96,[키즈/어린이],로미유 스토리[Romiyu Story],408만,10억6557만,433
97,[음악/댄스/가수],NCT DREAM,386만,3억7422만,228
98,[BJ/인물/연예인],허팝Heopop,381만,31억3892만,"1,847"


In [27]:
def convert_unit(s):
    s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개', '').replace(',', '')
    s = s.replace('만', '0000')
    return s
convert_unit('10억6557만')

'1065570000'

In [37]:
# page 1에서 10까지 크롤링하기
category_list, name_list, subscriber_list, view_list, video_list = [], [] ,[] , [], []
for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2) # 페이지가 로딩되기 전에 데이터를 가져오려고하면 오류가 발생할 수 있으니 해당 사항을 고려하여 대기시간 작성
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')
    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip()
        name = channel.select_one('.subject a').text.strip()
        subscriber = channel.select_one('.subscriber_cnt').text
        view = channel.select_one('.view_cnt').text
        video = channel.select_one('.video_cnt').text[:-1]
        category_list.append(category)
        name_list.append(name)
        subscriber_list.append(subscriber)
        view_list.append(view)
        video_list.append(video)
df = pd.DataFrame({
    '카테고리' : category_list, '이름' : name_list, '구독자 수' : subscriber_list, '조회수' : view_list, '영상 수' : video_list
})
df

,카테고리,이름,구독자 수,조회수,영상 수
0,[음악/댄스/가수],BLACKPINK,6400만,190억0381만,371
1,[음악/댄스/가수],HYBE LABELS,6030만,187억1305만,654
2,[음악/댄스/가수],BANGTANTV,5640만,121억9496만,"1,579"
3,[음악/댄스/가수],SMTOWN,2850만,218억7092만,"3,729"
4,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223
...,...,...,...,...,...
995,[게임],미소,51만,2억1077만,"3,084"
996,[미분류],밉지않은 관종언니,51만,6774만,170
997,[미분류],복지마블TV [Welfare Marble],51만,4996만,240
998,[음식/요리/레시피],Muggo,51만,1억8372만,"1,388"
